In [14]:
!pip install confluent_kafka

In [15]:
from confluent_kafka import Producer

In [32]:
broker = 'kafka-broker:9092'
import json
import tempfile
from minio import Minio
from datetime import datetime


In [33]:
def chamadaKafka(data,endpoint):
    broker = 'kafka-broker:9092'
    if(endpoint =="LI"):
        topic = 'api.Linhas'
    elif(endpoint=="PPL"):
        topic = 'api.ParadasPorLinha'
    else:
        topic='api.PrevisaoDeChegada'
    #print(data)
    
    def delivery_report(err, msg):
        if err is not None:
            print(f"Mensagem não entregue: {err}")
        else:
            print(f"Mensagem entregue para {msg.topic()} [{msg.partition()}]")

    # Criando o produtor Kafka
    producer = Producer({'bootstrap.servers': broker})

    # Serializando os dados para o formato JSON
    jsonKafka = json.dumps(data)

    # Enviando a mensagem para o Kafka
    producer.produce(topic, jsonKafka, callback=delivery_report)

    # Espera até que todas as mensagens pendentes sejam enviadas
    producer.flush()

    print(f"Mensagem enviada para o tópico {topic}")

In [34]:
def chamadaMinIO(datap,bucket,name,tipo,linha,parada):
    # Inicializando o cliente MinIO
    client = Minio("minio:9000", access_key="datalake", secret_key="datalake", secure=False)

    # Verificando se o bucket existe
    if client.bucket_exists(bucket):
        print(f"{bucket} existe")
    else:
        print(f"{bucket} não existe")
        client.make_bucket(bucket)
    # Dados a serem enviados (presumindo que você tem um dicionário em `data`)
    data = datap


    if isinstance(data, str):
        data = json.loads(data)

    # Criando um arquivo temporário para armazenar os dados JSON
    with tempfile.NamedTemporaryFile(delete=False, mode='w', newline='', encoding='utf-8') as tmpfile:
        # Escreve os dados JSON no arquivo temporário
        json.dump(data, tmpfile, ensure_ascii=False)  # ensure_ascii=False para manter caracteres especiais
        tmpfile_path = tmpfile.name  # Obtém o caminho do arquivo temporário
    
    if(tipo==1):
        object_name = f"Linhas/{name}_{linha}__{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.json"
        
    elif(tipo==2):
        object_name = f"Paradas/{name}_{linha}{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.json"
    
    else:
        object_name = f"Previsao/{name}_{linha}_{parada}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.json"
        
    # Enviar o arquivo temporário para o MinIO
    client.fput_object(bucket, object_name, tmpfile_path)

    # Exibindo sucesso
    print(f"Arquivo '{tmpfile_path}' enviado para o bucket '{bucket}' como '{object_name}'.")

    

In [37]:
import requests
import json

# Dados de autenticação
token = 'ef901a1e2d90324f90106392d4df1ed341f9cf6c40a3a2eaf4203ff074ce9a29'
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}

# Criando sessão
session = requests.Session()

# Realizando o login e autenticando
req = session.post(f'http://api.olhovivo.sptrans.com.br/v2.1/Login/Autenticar?token={token}', headers=headers)
response_data = req.json()

if req.status_code == 200:
    print("Autenticação bem-sucedida.")
else:
    print("Falha na autenticação.")
    exit()

linhas = ["8000","6000","6001","8001"]

paradas = []

################
# Busca Linhas #
################

i = 0
while i < len(linhas):
    # Realizando a busca pela linha
    req = session.get(f'http://api.olhovivo.sptrans.com.br/v2.1/Linha/Buscar?termosBusca={linhas[i]}')
    
    if req.status_code == 200:
        try:
            req = session.get(f'http://api.olhovivo.sptrans.com.br/v2.1/Linha/Buscar?termosBusca={linhas[i]}')
            data = json.dumps(req.json())
            dataParada= json.loads(data)
            chamadaKafka(dataParada, "LI")
                # Enviando dados para o MinIO
            chamadaMinIO(data, "raw", "Linha", 1, linhas[i], 0)
            
        except ValueError as e:
            print(f"Erro ao converter resposta para JSON: {e}")
    else:
        print(f"Falha na requisição de busca para a linha {linhas[i]}. Status code: {req.status_code}")

    i += 1
    
    data = json.loads(data)
    cl_values = [item['cl'] for item in data]
    
    for item in cl_values:
        paradas.append(item)
        


Autenticação bem-sucedida.
Mensagem entregue para api.Linhas [0]
Mensagem enviada para o tópico api.Linhas
raw existe
Arquivo '/tmp/tmp8y_r06zq' enviado para o bucket 'raw' como 'Linhas/Linha_8000__2025-02-05_17-37-23.json'.
Mensagem entregue para api.Linhas [0]
Mensagem enviada para o tópico api.Linhas
raw existe
Arquivo '/tmp/tmpzrxgqqri' enviado para o bucket 'raw' como 'Linhas/Linha_6000__2025-02-05_17-37-23.json'.
Mensagem entregue para api.Linhas [0]
Mensagem enviada para o tópico api.Linhas
raw existe
Arquivo '/tmp/tmpwg66rzng' enviado para o bucket 'raw' como 'Linhas/Linha_6001__2025-02-05_17-37-23.json'.
Mensagem entregue para api.Linhas [0]
Mensagem enviada para o tópico api.Linhas
raw existe
Arquivo '/tmp/tmpxv55c5hs' enviado para o bucket 'raw' como 'Linhas/Linha_8001__2025-02-05_17-37-23.json'.


In [38]:

#########################
# BuscarParadasPorLinha #
#########################

i=0
linhas = paradas

#linhas =[2506, 35274]
while (i<len(linhas)):
    print(linhas[i])
    req = session.post(f'http://api.olhovivo.sptrans.com.br/v2.1/Login/Autenticar?token={token}',headers=headers)
    response_data = req.json()
    #Criar uma sessão 
    if req.status_code == 200:
        req = session.get(f'http://api.olhovivo.sptrans.com.br/v2.1//Parada/BuscarParadasPorLinha?codigoLinha={linhas[i]}')
        data = json.dumps(req.json())
        dataParada = json.loads(data)
        chamadaKafka(dataParada,"PPL")
        chamadaMinIO(data, "raw", "ParadaPorLinha", 2, linhas[i], 0)
        data = json.loads(data)
        # Now you can use list comprehension to access 'cl'
        parada_values = [item['cp'] for item in data]
        print(parada_values)
        p=0
        #print(parada_values)
        
########################
# Previsão de chegada  #
########################   
        
        while(p<=len(parada_values)-1):
            req = session.get(f'http://api.olhovivo.sptrans.com.br/v2.1/Previsao?codigoParada={parada_values[p]}&codigoLinha={linhas[i]}')
            data = json.dumps(req.json())
            dataParada = json.loads(data)
            
            
            chamadaKafka(dataParada,"PC")
            chamadaMinIO(data, "raw", "PrevisaodeChegada", 3, linhas[i], parada_values[p])
            p=p+1
            
        
    else:
        print("Falha na autenticação.")
    
    i=i+1
    
print('Processo finalizado')

    
 

2506
Mensagem entregue para api.ParadasPorLinha [0]
Mensagem enviada para o tópico api.ParadasPorLinha
raw existe
Arquivo '/tmp/tmp0btpxegw' enviado para o bucket 'raw' como 'Paradas/ParadaPorLinha_25062025-02-05_17-37-34.json'.
[7014417, 60016784, 60016786, 60016788, 60016790, 480014612, 480014613, 480014614, 480014961, 610014603, 670010530, 700006086, 700016623, 700016791, 700016866]
Mensagem entregue para api.PrevisaoDeChegada [0]
Mensagem enviada para o tópico api.PrevisaoDeChegada
raw existe
Arquivo '/tmp/tmp9h32aku7' enviado para o bucket 'raw' como 'Previsao/PrevisaodeChegada_2506_7014417_2025-02-05_17-37-34.json'.
Mensagem entregue para api.PrevisaoDeChegada [0]
Mensagem enviada para o tópico api.PrevisaoDeChegada
raw existe
Arquivo '/tmp/tmp49me1cj_' enviado para o bucket 'raw' como 'Previsao/PrevisaodeChegada_2506_60016784_2025-02-05_17-37-34.json'.
Mensagem entregue para api.PrevisaoDeChegada [0]
Mensagem enviada para o tópico api.PrevisaoDeChegada
raw existe
Arquivo '/tmp/t